# Data Preprocessing

### Importazione librerie

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR


### Caricamento file train.csv

In [ ]:
file_path = "../../train.csv"
df = pd.read_csv(file_path)

### Controllo  e rimozione valori nulli e duplicati

In [ ]:
# Valori mancanti per ciascuna colonna
missing_values_per_column = df.isnull().sum()
print("\nValori mancanti nel DataFrame per ciascuna colonna:")
print(missing_values_per_column)

# Totale dei valori mancanti in tutto il DataFrame
total_missing_values = missing_values_per_column.sum()
print(f"\nTotale dei valori mancanti nel DataFrame:\n{total_missing_values}")

In [ ]:
print("\nDuplicati nel DataFrame:")
print(df.duplicated().sum())

In [ ]:
clean_df = df.dropna() 
clean_df = df.drop_duplicates()

In [ ]:
# Verifica rimozione valori nulli e duplicati
total_missing_values = missing_values_per_column.sum()
print(f"\nTotale dei valori mancanti nel DataFrame: {total_missing_values}")
print(f"\nDuplicati nel DataFrame: {clean_df.duplicated().sum()}")

# Rimozione outlier e visualizzazione post rimozione

In [ ]:
# Itera su ogni anno presente nel DataFrame
for year in clean_df['Year'].unique():
    # Filtra il DataFrame per l'anno corrente
    df_year = clean_df[clean_df['Year'] == year]
    
    # Calcola la mediana per ogni colonna
    median_values = df_year.median()
    
    # Calcola i limiti per individuare gli outlier per ogni colonna
    Q1 = df_year.quantile(0.25)
    Q3 = df_year.quantile(0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR
    
    # Sostituisci gli outlier con la mediana per ogni colonna
    def replace_outliers(row):
        for col in df_year.columns[1:]:  # Escludi la colonna 'year'
            if row[col] < lower_limit[col] or row[col] > upper_limit[col]:
                row[col] = median_values[col]
        return row
    
    df_year = df_year.apply(replace_outliers, axis=1) # Sostituzione outlier con mediana
    
    # Sostituisci i dati nel DataFrame originale
    clean_df.loc[clean_df['Year'] == year] = df_year

In [ ]:
# Seleziona solo le colonne che iniziano con 'S'
colonne_s = [col for col in clean_df.columns if col.startswith('S')]

# Lista per memorizzare i risultati originali e nuovi
risultati_outliers = []
risultati_nuovi_outliers = []

# Ciclo attraverso ogni colonna 'S'
for col in colonne_s:
    # Calcola i quantili e l'IQR (Interquartile Range) per i dati originali
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    
    # Calcola i baffi inferiori e superiori
    baffo_inferiore = q1 - 1.5 * iqr
    baffo_superiore = q3 + 1.5 * iqr
    
    # Trova gli outliers inferiori e superiori per i dati originali
    outliers_inferiori = df[df[col] < baffo_inferiore]
    outliers_superiori = df[df[col] > baffo_superiore]
    numero_outliers = len(outliers_inferiori) + len(outliers_superiori)
    
    # Calcola la percentuale di outliers
    percentuale_outliers = (numero_outliers / len(df)) * 100
    
    # Aggiungi i risultati alla lista
    risultati_outliers.append({
        'Colonna': col,
        'Numero outliers': numero_outliers,
        'Percentuale outliers': f'{percentuale_outliers:.2f}%'
    })
    
    # Calcola i quantili e l'IQR (Interquartile Range) per i dati modificati
    q1_new = clean_df[col].quantile(0.25)
    q3_new = clean_df[col].quantile(0.75)
    iqr_new = q3_new - q1_new
    
    # Calcola i baffi inferiori e superiori per i dati modificati
    baffo_inferiore_new = q1_new - 1.5 * iqr_new
    baffo_superiore_new = q3_new + 1.5 * iqr_new
    
    # Trova gli outliers inferiori e superiori per i dati modificati
    outliers_inferiori_new = clean_df[clean_df[col] < baffo_inferiore_new]
    outliers_superiori_new = clean_df[clean_df[col] > baffo_superiore_new]
    numero_outliers_new = len(outliers_inferiori_new) + len(outliers_superiori_new)
    
    # Calcola la percentuale di nuovi outliers
    percentuale_outliers_new = (numero_outliers_new / len(clean_df)) * 100
    
    # Aggiungi i risultati alla lista dei nuovi outliers
    risultati_nuovi_outliers.append({
        'Colonna': col,
        'Numero nuovi outliers': numero_outliers_new,
        'Percentuale nuovi outliers': f'{percentuale_outliers_new:.2f}%'
    })

# Converti le liste in DataFrame
df_outliers = pd.DataFrame(risultati_outliers)
df_nuovi_outliers = pd.DataFrame(risultati_nuovi_outliers)

# Unisci i DataFrame sui risultati originali e nuovi
outliers_summary_df = df_outliers.merge(df_nuovi_outliers, on='Colonna')

# Visualizza la tabella finale
print(outliers_summary_df)


### Suddivisione del DataSet

In [ ]:
X = clean_df.drop(columns=['Year'])  # Non la consideriamo per il modello
y = clean_df['Year'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

### Standardizzazione

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Salva il modello di scaling
with open('scaler.save', 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
training_set_size = X_train.shape[0]
test_set_size = X_test.shape[0]
total_size = X.shape[0]

print(f"Dimensione del training set: {training_set_size} ({(training_set_size / total_size) * 100:.2f}% del dataset totale)")
print(f"Dimensione del test set: {test_set_size} ({(test_set_size / total_size) * 100:.2f}% del dataset totale)")
# print("\nDati di training normalizzati:")
# print(X_train_scaled)

### PCA

In [ ]:
X_rf = clean_df.drop('Year', axis=1)
y_rf = clean_df['Year']

In [ ]:
# Riduzione della dimensionalità con PCA
pca = PCA(n_components=0.95)  # Mantieni il 95% della varianza spiegata
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"Numero di componenti principali: {pca.n_components_}")
print(f"Forma dei dati di training dopo PCA: {X_train_pca.shape}")
print(f"Forma dei dati di test dopo PCA: {X_test_pca.shape}") 

# Modeling

In [ ]:
# Calcolo e stampe metriche di valutazione del modello
def valutazione_modello(y_pred, y_true):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)
    
    print(f"MSE (Mean Squared Error): {mse}")
    print(f"MAE (Mean Absolute Error): {mae}")
    print(f"MAPE (Mean Absolute Percentage Error): {mape}%")
    print(f"R-squared: {r2}")
    #return mse, mae, mape, r2

## Random Forest

In [ ]:
# Definizione dei parametri per il tuning
param_grid_rf = {
    'n_estimators': [300, 400, 450],
    'max_depth': [140, 160, 170], 
    'min_samples_split': [3, 5, 8],
    'min_samples_leaf': [3, 5] 
}

# Creazione del modello e GridSearchCV
rf = RandomForestRegressor(random_state=42)
grid_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, scoring='neg_mean_squared_error')
grid_rf.fit(X_train_scaled, y_train)

# Migliori parametri trovati per Random Forest
print(f"Migliori parametri trovati per Random Forest: {grid_rf.best_params_}")

# Predizione
y_pred_rf = grid_rf.best_estimator_.predict(X_test_scaled)

# Valutazione del modello
print("Risultati random forest:")
valutazione_modello(y_pred_rf, y_test)

In [ ]:
# Set modello con migliori iperparametri
rf = RandomForestRegressor(
    n_estimators=450,
    max_depth=170,
    min_samples_split=8,
    min_samples_leaf=5,
)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit del modello su tutto il df scalato
rf.fit(X_scaled, y)

# Salvataggio del modello su disco
with open("rf.save", "wb") as file:
    pickle.dump(rf, file)

In [ ]:
# Previsione sui dati di test con il modello RF già addestrato
y_pred_rf = grid_rf.best_estimator_.predict(X_test_scaled)

# Creazione del grafico Valori previsione vs Valori reali
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_rf, color='blue', alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2)  # Linea diagonale
plt.xlabel('Valori reali')
plt.ylabel('Valori predetti')
plt.title('RF: valori previsione vs valori reali')
plt.grid(True)
plt.show()

## LR

In [ ]:
linear_regr = LinearRegression()
# Addestriamo modello sui dati di training
linear_regr.fit(X_train_scaled, y_train)

# Predizioni sul set di test
y_pred_lr = linear_regr.predict(X_test_scaled)

# Valutazione del modello
print("Risultati linear regression:")
valutazione_modello(y_pred_lr, y_test)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

linear_model = LinearRegression()
linear_model.fit(X_scaled, y) # Fit su tutto il df

# Salvataggio del modello su disco
with open("lr.save", "wb") as file:
    pickle.dump(linear_model, file)

In [ ]:
# Previsione sui dati di test con il modello Linear Regression già addestrato
y_pred_lr = linear_regr.predict(X_test_scaled)

# Creazione del grafico Valori previsione vs Valori reali per Linear Regression
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_lr, color='blue', alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2)  # Linea diagonale
plt.xlabel('Valori reali')
plt.ylabel('Valori predetti')
plt.title('Linear Regression: valori previsione vs valori reali')
plt.grid(True)
plt.show()

## SVR

In [ ]:
# Definizione dei parametri per il tuning
param_grid_svr = {
    'C': [ 0.1, 0.4, 0.8],
    'kernel': [ 'sigmoid', 'rbf',  'linear', 'poly' ],
    'gamma': [ 'scale', 0.5 ],
    'epsilon' : [ 0.2, 0.4, 0.5 ] 
}

# Creazione del modello e GridSearchCV
svr = SVR()
grid_svr = GridSearchCV(estimator=svr, param_grid=param_grid_svr, cv=5, scoring='neg_mean_squared_error')
grid_svr.fit(X_train_scaled, y_train)

# Migliori parametri trovati per SVR
print(f"Migliori parametri trovati per SVR: {grid_svr.best_params_}")

# Predizione
y_pred_svr = grid_svr.best_estimator_.predict(X_test_scaled)

# Valutazione del modello
print("Risultati SVR:")
valutazione_modello(y_pred_svr, y_test)

In [ ]:
# Set migliori parametri 
best_params = {
    'C': 0.4,
    'kernel': 'rbf',
    'gamma': 'scale',
    'epsilon': 0.5
}

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Inizializza il modello SVR con i migliori iperparametri
svr = SVR(**best_params)

# Addestramento del modello
svr.fit(X_scaled, y)

# Salvataggio del modello su disco
with open("svr.save", "wb") as file:
    pickle.dump(svr, file)

In [ ]:
# Previsione sui dati di test con il modello SVR già addestrato
y_pred_svr = grid_svr.best_estimator_.predict(X_test_scaled)

# Creazione del grafico Valori previsione vs Valori reali
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_svr, color='blue', alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2)  # Linea diagonale
plt.xlabel('Valori reali')
plt.ylabel('Valori predetti')
plt.title('SVR: valori previsione vs valori reali')
plt.grid(True)
plt.show()

## KNN

In [ ]:
# Definizione dei parametri per il tuning
param_grid_knn = {
    'n_neighbors': [5, 10, 15, 17, 30, 35, 40],
    'weights': ['distance', 'uniform'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Creazione del modello e GridSearchCV
knn = KNeighborsRegressor()
grid_knn = GridSearchCV(estimator=knn, param_grid=param_grid_knn, cv=5, scoring='neg_mean_absolute_error')
grid_knn.fit(X_train_scaled, y_train)

# Migliori parametri trovati per KNN
print(f"Migliori parametri trovati per KNN: {grid_knn.best_params_}")

# Predizione
y_pred_knn = grid_knn.best_estimator_.predict(X_test_scaled)

# Valutazione del modello
print("Risultati KNN:")
valutazione_modello(y_pred_knn, y_test)

In [ ]:
# Migliori parametri ottenuti
best_params = {
    'n_neighbors': 17,
    'weights': 'distance',
    'metric': 'manhattan'
}

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Inizializza il modello KNN con i migliori iperparametri
knn = KNeighborsRegressor(**best_params)

# Addestramento del modello
knn.fit(X_scaled, y)

# Salvataggio del modello su disco
with open("knn.save", "wb") as file:
    pickle.dump(knn, file)

In [ ]:
# Previsione sui dati di test con il modello KNN già addestrato
y_pred_knn = grid_knn.best_estimator_.predict(X_test_scaled)

# Creazione del grafico Valori previsione vs Valori reali
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_knn, color='blue', alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2)  # Linea diagonale
plt.xlabel('Valori reali')
plt.ylabel('Valori predetti')
plt.title('KNN: valori previsione vs valori reali')
plt.grid(True)
plt.show()